In [42]:
class TSAFM_Ordinances:
    """
    Implementation of a Transition-Assigned Finite State Machine (Mealy Automaton)

    where:
    Q - Set of states
    S - input alphabet
    R - output alphabet
    f - state transition function (Q x S -> Q)
    g - output function (Q x S -> R)
    qi - initial state
    """

    def __init__(self, Q, S, R, f, g, qi):
        self.Q = Q
        self.S = S
        self.R = R
        self.f = f
        self.g = g
        self.q0 = qi

    def run(self, input_str):
        """
        Processes an input string and returns the output generated by the FSM.

        Returns:
        - output: the output generated by the FSM for the input string
        """
        q = self.q0
        ordinance_codes = []
        candidate_string = ""
        for symbol_raw in input_str.split(" "):
            token = None
            symbol = symbol_raw.lower()

            # Check if symbol is reserved word
            if symbol in S:
                token = symbol
            # Check if symbol is valid number candidate
            elif len(symbol) > 1:
                # Checks if the symbol is a number, while taking into consideration a 
                # possible trailing comma or period in the string
                if (symbol.isdigit() or symbol[:-1].isdigit()) and symbol[-2].isdigit():
                    token = "_%number%_"

            # If there is no next state given an current state and input,
            # Then go back to state A and reset candidate_string
            q_next = self.f.get((q, token), None)
            if q_next is None:
                q = "A"
                candidate_string = ""
                continue

            # If the next state is A, this means that the candidate string 
            # is now a complete and valid ordinance code.
            # Append candidate_string to the list of ordinance codes taken
            # from given input.
            # Reset candidate string to empty string
            if q_next == "A":
                # Remove trailing comma or period in year if there is any
                if symbol[-1] == "," or symbol[-1] == ".":
                    symbol_raw = symbol_raw[:-1]
                candidate_string += symbol_raw + " "
                ordinance_codes.append(candidate_string.strip())
                candidate_string = ""
                q = q_next
                pass

            # If there is a new state, then append current word to 
            # candidate_string
            candidate_string += symbol_raw + " "
            q = q_next

        return ordinance_codes


"""
Sample FSM: Accepts Valid Ordinance Codes
"""
# Define the parameters for the TSAFM
Q = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}
S = {'ordinance', 'no.', '_%number%_', 'series', 'of', ''}
R = {'1', '0'}
f = {('A', 'ordinance'): 'B', ('B', 'no.'): 'C', ('C', '_%number%_'): 'D', ('D', 'series'): 'E', ('E', 'of'): 'G', ('G', '_%number%_'): 'A'}
g = {('A', 'ordinance'): '1', ('B', 'no.'): '1', ('C', '_%number%_'): '1', ('D', 'series'): '1', ('E', 'of'): '1', ('G', '_%number%_'): '1'}
qi = 'A'

# Create an instance of the TSAFM
fsm = TSAFM_Ordinances(Q, S, R, f, g, qi)

extracted_ordinances = fsm.run("Mayor Benjamin Magalong signed city ordinance no. 26, series of 2023, approving the request for authority to charge the amount of P28,000 against the 2023 current appropriations of the City Administrator’s Office (CAO)  for the payment of tokens of personalized eco-bag purchased since 2020. The quick brown Ordinance nO. 26,a Ordinance series of 2023 The quick brown ordinance no. 26,  asfasfas fseries of 2023 The quick brown ORdInance no. 26, Series oF 2023")

for ordinance in extracted_ordinances:
    print(ordinance)

ordinance no. 26, series of 2023
ORdInance no. 26, Series oF 2023
